In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow_datasets as tfds
import datetime

2024-09-20 03:16:40.320820: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-20 03:16:42.262902: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-20 03:16:42.272356: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-20 03:16:46.636915: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/gilles/.pyenv/versions/3.8.12/envs/nlp05env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the IMDB dataset using TensorFlow Datasets
(ds_train, ds_test), ds_info = tfds.load('imdb_reviews', split=['train', 'test'], as_supervised=True, with_info=True)


In [5]:
ds = (ds_train, ds_test)

In [10]:
#convert a tensor flow dataset to a dataframe
x = tfds.as_dataframe(ds_test.take(4), ds_info)
x

2024-09-20 03:33:46.383428: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


,label,text
0,1 (pos),"There are films that make careers. For George Romero, it was NIGHT OF THE LIVING DEAD; for Kevin Smith, CLERKS; for Robert Rodriguez, EL MARIACHI. Add to that list Onur Tukel's absolutely amazing DING-A-LING-LESS. Flawless film-making, and as assured and as professional as any of the aforementioned movies. I haven't laughed this hard since I saw THE FULL MONTY. (And, even then, I don't think I laughed quite this hard... So to speak.) Tukel's talent is considerable: DING-A-LING-LESS is so chock full of double entendres that one would have to sit down with a copy of this script and do a line-by-line examination of it to fully appreciate the, uh, breadth and width of it. Every shot is beautifully composed (a clear sign of a sure-handed director), and the performances all around are solid (there's none of the over-the-top scenery chewing one might've expected from a film like this). DING-A-LING-LESS is a film whose time has come."
1,1 (pos),"A blackly comic tale of a down-trodden priest, Nazarin showcases the economy that Luis Bunuel was able to achieve in being able to tell a deeply humanist fable with a minimum of fuss. As an output from his Mexican era of film making, it was an invaluable talent to possess, with little money and extremely tight schedules. Nazarin, however, surpasses many of Bunuel's previous Mexican films in terms of the acting (Francisco Rabal is excellent), narrative and theme.<br /><br />The theme, interestingly, is something that was explored again in Viridiana, made three years later in Spain. It concerns the individual's struggle for humanity and altruism amongst a society that rejects any notion of virtue. Father Nazarin, however, is portrayed more sympathetically than Sister Viridiana. Whereas the latter seems to choose charity because she wishes to atone for her (perceived) sins, Nazarin's whole existence and reason for being seems to be to help others, whether [...]"
2,0 (neg),"Scary Movie 1-4, Epic Movie, Date Movie, Meet the Spartans, Not another Teen Movie and Another Gay Movie. Making ""Superhero Movie"" the eleventh in a series that single handily ruined the parody genre. Now I'll admit it I have a soft spot for classics such as Airplane and The Naked Gun but you know you've milked a franchise so bad when you can see the gags a mile off. In fact the only thing that might really temp you into going to see this disaster is the incredibly funny but massive sell-out Leslie Neilson.<br /><br />You can tell he needs the money, wither that or he intends to go down with the ship like a good Capitan would. In no way is he bringing down this genre but hell he's not helping it. But if I feel sorry for anybody in this film its decent actor Drake Bell who is put through an immense amount of embarrassment. The people who are put through the largest amount of torture by far however is the audience forced to sit through 90 [...]"
3,0 (neg),"Poor Shirley MacLaine tries hard to lend some gravitas to this mawkish, gag-inducing ""feel-good"" movie, but she's trampled by the run-away sentimentality of a film that's not the least bit grounded in reality.<br /><br />This was directed by Curtis Hanson? Did he have a lobotomy since we last heard from him? Hanson can do effective drama sprinkled with comedy, as evidenced by ""Wonder Boys."" So I don't know what happened to him here. This is the kind of movie that doesn't want to accept that life is messy and fussy, and that neat, tidy endings (however implausible they might be) might make for a nice closing shot, but come across as utterly phony if the people watching the film have been through anything remotely like what the characters in the film go through.<br /><br />My wife and I made a game of calling out the plot points before they occurred -- e.g. ""the old man's going to teach her to read and then [...]"


In [5]:
# Load the IMDB dataset using TensorFlow Datasets
(ds_train, ds_test), ds_info = tfds.load('imdb_reviews', split=['train', 'test'], as_supervised=True, with_info=True)

# Split the training data into train and validation sets
def train_val_split(ds, val_size=0.2):
    ds = ds.shuffle(10000, seed=42)
    val_size = int(val_size * ds_info.splits['train'].num_examples)
    ds_val = ds.take(val_size)
    ds_train = ds.skip(val_size)
    return ds_train, ds_val

ds_train, ds_val = train_val_split(ds_train)

# Batch and prefetch the datasets for performance
batch_size = 32
ds_train = ds_train.batch(batch_size).prefetch(tf.data.AUTOTUNE)
ds_val = ds_val.batch(batch_size).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(batch_size).prefetch(tf.data.AUTOTUNE)# why ?

# Load the preprocessor and ALBERT model from TensorFlow Hub
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/albert_en_preprocess/3")
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/albert_en_base/3")

# Build the model
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = preprocessor(text_input)
encoder_outputs = encoder(preprocessed_text)
pooled_output = encoder_outputs['pooled_output']
output = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(pooled_output)

model = tf.keras.Model(inputs=[text_input], outputs=[output])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 3
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

model.fit(ds_train, validation_data=ds_val, epochs=epochs, callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(ds_test)
print(f"Test Accuracy: {accuracy:.4f}")


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: [Errno 9] Bad file descriptor
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: [Errno 9] Bad file descriptor
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: [Errno 9] Bad file descriptor
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Epoch 1/3


In [4]:
# Load the IMDB dataset using TensorFlow Datasets
(ds_train, ds_test), ds_info = tfds.load('imdb_reviews', split=['train', 'test'], as_supervised=True, with_info=True)

# Split the training data into train and validation sets
def train_val_split(ds, val_size=0.2):
    ds = ds.shuffle(10000, seed=42)
    val_size = int(val_size * ds_info.splits['train'].num_examples)
    ds_val = ds.take(val_size)
    ds_train = ds.skip(val_size)
    return ds_train, ds_val

ds_train, ds_val = train_val_split(ds_train)

# Reduce batch size to lessen memory usage
batch_size = 16
ds_train = ds_train.batch(batch_size).prefetch(tf.data.AUTOTUNE)
ds_val = ds_val.batch(batch_size).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Load the preprocessor and ALBERT model from TensorFlow Hub
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/albert_en_preprocess/3")
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/albert_en_base/3")

# Build the model
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = preprocessor(text_input)
encoder_outputs = encoder(preprocessed_text)
pooled_output = encoder_outputs['pooled_output']
output = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(pooled_output)

model = tf.keras.Model(inputs=[text_input], outputs=[output])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Enable mixed precision (if GPU supports it)
from tensorflow.keras.mixed_precision import set_global_policy
policy = tf.keras.mixed_precision.Policy('mixed_float16')
set_global_policy(policy)

# Train the model
epochs = 3
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

model.fit(ds_train, validation_data=ds_val, epochs=epochs, callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(ds_test)
print(f"Test Accuracy: {accuracy:.4f}")


The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


Epoch 1/3
1250/1250 [==============================] - 8140s 6s/step - loss: 0.7121 - accuracy: 0.4816 - val_loss: 0.7074 - val_accuracy: 0.4754
Epoch 2/3
 354/1250 [=======>......................] - ETA: 1:09:34 - loss: 0.7028 - accuracy: 0.4963

In [2]:
# Load the IMDB dataset using TensorFlow Datasets
(ds_train, ds_test), ds_info = tfds.load('imdb_reviews', split=['train', 'test'], as_supervised=True, with_info=True)

# Split the training data into train and validation sets
def train_val_split(ds, val_size=0.2):
    ds = ds.shuffle(10000, seed=42)
    val_size = int(val_size * ds_info.splits['train'].num_examples)
    ds_val = ds.take(val_size)
    ds_train = ds.skip(val_size)
    return ds_train, ds_val

ds_train, ds_val = train_val_split(ds_train)

# Reduce batch size to lessen memory usage
batch_size = 16
ds_train = ds_train.batch(batch_size).prefetch(tf.data.AUTOTUNE)
ds_val = ds_val.batch(batch_size).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Load the preprocessor and ALBERT model from TensorFlow Hub
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/albert_en_preprocess/3")
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/albert_en_base/3")

# Build the model
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = preprocessor(text_input)
encoder_outputs = encoder(preprocessed_text)
pooled_output = encoder_outputs['pooled_output']
output = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(pooled_output)

model = tf.keras.Model(inputs=[text_input], outputs=[output])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 3
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

model.fit(ds_train, validation_data=ds_val, epochs=epochs, callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(ds_test)
print(f"Test Accuracy: {accuracy:.4f}")


Epoch 1/3
1250/1250 [==============================] - 8155s 7s/step - loss: 0.6989 - accuracy: 0.5001 - val_loss: 0.6972 - val_accuracy: 0.5040
Epoch 2/3
1250/1250 [==============================] - 8309s 7s/step - loss: 0.6937 - accuracy: 0.5191 - val_loss: 0.6891 - val_accuracy: 0.5424
Epoch 3/3
1563/1563 [==============================] - 8053s 5s/step - loss: 0.6840 - accuracy: 0.5554
Test Accuracy: 0.5554


In [3]:
#### SMALLER BERT MODEL#####

In [4]:
# Load the IMDB dataset using TensorFlow Datasets
(ds_train, ds_test), ds_info = tfds.load('imdb_reviews', split=['train', 'test'], as_supervised=True, with_info=True)

# Split the training data into train and validation sets
def train_val_split(ds, val_size=0.2):
    ds = ds.shuffle(10000, seed=42)
    val_size = int(val_size * ds_info.splits['train'].num_examples)
    ds_val = ds.take(val_size)
    ds_train = ds.skip(val_size)
    return ds_train, ds_val

ds_train, ds_val = train_val_split(ds_train)

# Reduce batch size to lessen memory usage
batch_size = 16
ds_train = ds_train.batch(batch_size).prefetch(tf.data.AUTOTUNE)
ds_val = ds_val.batch(batch_size).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Load the preprocessor and DistilBERT model from TensorFlow Hub
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2")

# Build the model
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = preprocessor(text_input)
encoder_outputs = encoder(preprocessed_text)
pooled_output = encoder_outputs['pooled_output']
output = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(pooled_output)

model = tf.keras.Model(inputs=[text_input], outputs=[output])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 3
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

model.fit(ds_train, validation_data=ds_val, epochs=epochs, callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(ds_test)
print(f"Test Accuracy: {accuracy:.4f}")


Epoch 1/3
1250/1250 [==============================] - 143s 109ms/step - loss: 0.7052 - accuracy: 0.5330 - val_loss: 0.6974 - val_accuracy: 0.5468
Epoch 2/3
1250/1250 [==============================] - 135s 108ms/step - loss: 0.7009 - accuracy: 0.5345 - val_loss: 0.7033 - val_accuracy: 0.5282
Epoch 3/3
1563/1563 [==============================] - 137s 87ms/step - loss: 0.6955 - accuracy: 0.5432
Test Accuracy: 0.5432


In [3]:
# Load the IMDB dataset using TensorFlow Datasets
(ds_train, ds_test), ds_info = tfds.load('imdb_reviews', split=['train', 'test'], as_supervised=True, with_info=True)


In [4]:
ds_train

<_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [11]:
x1 = tfds.as_dataframe(ds_train.take(4), ds_info)
x1

2024-09-20 03:39:14.977090: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


,label,text
0,0 (neg),"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
1,0 (neg),"I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of what was causing them or why. I admit, I may have missed part of the film, but i watched the majority of it and everything just seemed to happen of its own accord without any real concern for anything else. I cant recommend this film at all."
2,0 (neg),"Mann photographs the Alberta Rocky Mountains in a superb fashion, and Jimmy Stewart and Walter Brennan give enjoyable performances as they always seem to do. <br /><br />But come on Hollywood - a Mountie telling the people of Dawson City, Yukon to elect themselves a marshal (yes a marshal!) and to enforce the law themselves, then gunfighters battling it out on the streets for control of the town? <br /><br />Nothing even remotely resembling that happened on the Canadian side of the border during the Klondike gold rush. Mr. Mann and company appear to have mistaken Dawson City for Deadwood, the Canadian North for the American Wild West.<br /><br />Canadian viewers be prepared for a Reefer Madness type of enjoyable howl with this ludicrous plot, or, to shake your head in disgust."
3,1 (pos),"This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received."


In [6]:

# Split the training data into train and validation sets
def train_val_split(ds, val_size=0.2):
    ds = ds.shuffle(10000, seed=42)
    val_size = int(val_size * ds_info.splits['train'].num_examples)
    ds_val = ds.take(val_size)
    ds_train = ds.skip(val_size)
    return ds_train, ds_val

In [7]:
ds_train, ds_val = train_val_split(ds_train)

In [8]:
# Reduce batch size to lessen memory usage
batch_size = 16
ds_train = ds_train.batch(batch_size).prefetch(tf.data.AUTOTUNE)
ds_val = ds_val.batch(batch_size).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [8]:
# Load the preprocessor and DistilBERT model from TensorFlow Hub
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2")


In [12]:
# Build the model
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = preprocessor(text_input)
encoder_outputs = encoder(preprocessed_text)
pooled_output = encoder_outputs['pooled_output']
output = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(pooled_output)

model = tf.keras.Model(inputs=[text_input], outputs=[output])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), loss='binary_crossentropy', metrics=['accuracy'])


In [13]:
# Train the model
epochs = 3
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

model.fit(ds_train, validation_data=ds_val, epochs=epochs, callbacks=[early_stopping])

Epoch 1/3
1250/1250 [==============================] - 177s 136ms/step - loss: 1.1266 - accuracy: 0.4983 - val_loss: 0.8518 - val_accuracy: 0.5000
Epoch 2/3
1250/1250 [==============================] - 197s 157ms/step - loss: 0.7511 - accuracy: 0.5056 - val_loss: 0.7078 - val_accuracy: 0.4996
Epoch 3/3
1250/1250 [==============================] - 192s 152ms/step - loss: 0.7029 - accuracy: 0.5070 - val_loss: 0.6974 - val_accuracy: 0.5140


In [14]:
# Evaluate the model
loss, accuracy = model.evaluate(ds_test)
print(f"Test Accuracy: {accuracy:.4f}")

1563/1563 [==============================] - 171s 109ms/step - loss: 0.6955 - accuracy: 0.5234
Test Accuracy: 0.5234


In [ ]:
#Exemple of pre and postprocessed_text

for text, label in ds_train.take(5):
    preprocessed_text = preprocessor(text)
    print(f'Original Text: {text.numpy()}')
    print(f'Preprocessed Text: {preprocessed_text}')

In [9]:
# Configure TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


In [21]:
# train model
model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=epochs,
    callbacks=[early_stopping, tensorboard_callback]
)


Epoch 1/3
1250/1250 [==============================] - ETA: 0s - loss: 0.6954 - accuracy: 0.5225

2024-08-29 01:22:18.794044: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 937635840 exceeds 10% of free system memory.


1250/1250 [==============================] - 165s 131ms/step - loss: 0.6954 - accuracy: 0.5225 - val_loss: 0.6917 - val_accuracy: 0.5350
Epoch 2/3
1250/1250 [==============================] - ETA: 0s - loss: 0.6917 - accuracy: 0.5343

2024-08-29 01:25:37.970866: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 937635840 exceeds 10% of free system memory.


1250/1250 [==============================] - 159s 126ms/step - loss: 0.6917 - accuracy: 0.5343 - val_loss: 0.6879 - val_accuracy: 0.5428
Epoch 3/3
1250/1250 [==============================] - ETA: 0s - loss: 0.6872 - accuracy: 0.5456

2024-08-29 01:28:20.978113: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 937635840 exceeds 10% of free system memory.


1250/1250 [==============================] - 162s 129ms/step - loss: 0.6872 - accuracy: 0.5456 - val_loss: 0.6843 - val_accuracy: 0.5530


In [22]:
# Evaluate model
loss, accuracy = model.evaluate(ds_test)
print(f"Test Accuracy: {accuracy:.4f}")

1563/1563 [==============================] - 148s 94ms/step - loss: 0.6829 - accuracy: 0.5604
Test Accuracy: 0.5604


In [ ]:
# Load TensorBoard in the notebook
 %load_ext tensorboard
%tensorboard --logdir logs/fit

 Change with a another model to see if the accuracy improve

In [ ]:
# Load the preprocessor and DistilBERT model from TensorFlow Hub
#preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
#encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2")

# choose a model from TensorFlow Hub
# use base BERT model
# preprocessor_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
# encoder_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3"


# use base ALBERT model
preprocessor_url ="https://tfhub.dev/tensorflow/albert_en_preprocess/3"
encoder_url ="https://tfhub.dev/tensorflow/albert_en_base/3"

# load preprocesor and encodor
preprocessor = hub.KerasLayer(preprocessor_url)
encoder = hub.KerasLayer(encoder_url)

# Build the model
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = preprocessor(text_input)
encoder_outputs = encoder(preprocessed_text)
pooled_output = encoder_outputs['pooled_output']
output = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(pooled_output)

model = tf.keras.Model(inputs=[text_input], outputs=[output])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 3
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

model.fit(ds_train, validation_data=ds_val, epochs=epochs, callbacks=[early_stopping, tensorboard_callback])

# Evaluate the model
loss, accuracy = model.evaluate(ds_test)
print(f"Test Accuracy: {accuracy:.4f}")


SST-2 (Stanford Sentiment Treebank) for sentiment analysis

Binary classification: positive/negative sentiment labels
Sentences from movie reviews
11,855 instances
Fully labeled parse trees for compositional sentiment analysis

In [19]:
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import tensorflow as tf

In [52]:
# Load the SST-2 dataset without `as_supervised=True`
dataset, info = tfds.load('glue/sst2', with_info=True, batch_size=-1, as_supervised=False)


In [ ]:
# Print the dataset info
print("Dataset Info:")
print(info)

# Print specific details about the dataset
print("\nFeatures:")
print(info.features)

print("\nSplits:")
for split, split_info in info.splits.items():
    print(f"{split}: {split_info}")

print("\nDescription:")
print(info.description)

print("\nHomepage:")
print(info.homepage)


In [55]:
# Extract data and labels
train_data = dataset['train']
val_data = dataset['validation']

# Extract texts and labels from the dataset
train_texts = train_data['sentence'].numpy()
train_labels = train_data['label'].numpy()
val_texts = val_data['sentence'].numpy()
val_labels = val_data['label'].numpy()

In [58]:
print("Train texts type:", type(train_texts))
print("Train labels type:", type(train_labels))
print("Val texts type:", type(val_texts))
print("Val labels type:", type(val_labels))
print("Train texts shape:", train_texts.shape)
print("Train labels shape:", train_labels.shape)


Train texts type: <class 'numpy.ndarray'>
Train labels type: <class 'numpy.ndarray'>
Val texts type: <class 'numpy.ndarray'>
Val labels type: <class 'numpy.ndarray'>
Train texts shape: (67349,)
Train labels shape: (67349,)


Batching at fit step

In [15]:
# Load the IMDB dataset using TensorFlow Datasets
(ds_train, ds_test), ds_info = tfds.load('imdb_reviews', split=['train', 'test'], as_supervised=True, with_info=True)


In [18]:
x0= tfds.as_dataframe(ds_train.take(4), ds_info)
x0

2024-09-20 04:06:40.817285: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


,label,text
0,1 (pos),"Australia's first mainstream slasher film hits the screen with a bang. And a stab. And a slice. And a scream or two. And plenty of blood, frights, red herrings and lots of laughs.<br /><br />In fact, there's lots of first surrounding Cut - it's the first script of Dave Warner's to be produced, although he has several others either optioned or in negotiation; it's the first major film from director and former Hoodoo Guru Kimble Rendall; and it's also the first film for producer Martin Fabinyi. And for a bunch of guys dipping their toes into this genre for the first time, they sure know their stuff.<br /><br />Cut tells the story of a bunch of Australian film students who hear about a slasher film, Hot Blooded, that was never finished because its director, Hilary (Kylie Minogue), was killed by the actor playing the psycho killer in the film.<br /><br />Despite their lecturer (who was assistant director on the night [...]"
1,0 (neg),"Yes, In 35 years of film going I have finally viewed the stinker that surpasses all other ghastly movies I have seen. Beating 'Good Will Hunting' Baise Moi' and 'Flirt' for sheer awfulness. This is pretentious blige of the first order... not even entertaining pretentious bilge. The effects are cheap, and worse - pointless.<br /><br />The script seems to have been written by a first year film student who doesn't get out much but wants to appear full of portent! The acting is simply undescribably bad - Tilda Swinton caps a career filled with vacuous woodeness with a performance which veers neurotically between comotose and laughable 'intensity'. Apparently, some fool out there has allowed the director of this film to make another one... be warned"
2,1 (pos),"I'm a pretty old dude, old enough to remember the taste of Oreos and Coke as they were 50-55 years ago, when every taste for a kid was fresh. I wish I have somehow set some aside then is some magical suspended locker, so that I could taste those things today. This magical locker might even have adjusted the fabric of the food to account for how I've drifted, physically and otherwise, a sort of dynamic chemistry of expectations. Over the half century, they would have had to adjust quite a bit, because you see I would have known that I set them aside. Eating one now would be a celebration of self and past, and story, and sense that would almost make the intervening years an anticipated reward.<br /><br />I didn't have enough sense to do that with original Coke. And I couldn't have invented one of those magical psychic lockers  not then. But I did something almost as good. In the seventies, I really tuned into Roman Polanski. He was a strange and [...]"
3,1 (pos),"Miriam Hopkins is ""The Lady with Red Hair"" in this 1940 biopic of Mrs. Leslie Carter which also stars Claude Rains as David Belasco, Richard Ainley as Lou Payne, and a fine cast of supporting players, including Laura Hope Crews and Victor Jory.<br /><br />Miriam Hopkins and Claude Rains give wonderful performances. Hopkins was a beautiful actress who really makes us feel for Mrs. Carter. Rains is great as the flamboyant, egotistical producer/writer/actor/impresario David Belasco, one of the great names in theater.<br /><br />Though Mrs. Carter's second husband, Lou Payne, served as adviser on this film, it's a poor representation of the real events of Mrs. Carter's life. True, there was a much publicized and bitter divorce, and she was undoubtedly viewed as a scandalous character for that and for becoming an actress. However, she had custody of her son Dudley, so there was no custody battle. Once she broke with Belasco, she did not [...]"


In [16]:
# Split the training data into train and validation sets
def train_val_split(ds, val_size=0.2):
    ds = ds.shuffle(10000, seed=42)
    val_size = int(val_size * ds_info.splits['train'].num_examples)
    ds_val = ds.take(val_size)
    ds_train = ds.skip(val_size)
    return ds_train, ds_val

ds_train, ds_val = train_val_split(ds_train)

In [ ]:
x2 = tfds.as_dataframe(ds_train.take(4), ds_info)
x2

In [19]:
# Load the preprocessor and DistilBERT model from TensorFlow Hub
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2")


In [20]:
# Build the model
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = preprocessor(text_input)
encoder_outputs = encoder(preprocessed_text)
pooled_output = encoder_outputs['pooled_output']
output = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(pooled_output)

In [21]:
model = tf.keras.Model(inputs=[text_input], outputs=[output])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), loss='binary_crossentropy', metrics=['accuracy'])


In [24]:
# Batch size provided directly in fit function
batch_size = 32
model.fit(ds_train, validation_data=ds_val, epochs=epochs, callbacks=[early_stopping], batch_size=batch_size)


Epoch 1/3


ValueError: in user code:

    File "/home/gilles/.pyenv/versions/3.8.12/envs/nlp05env/lib/python3.8/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/home/gilles/.pyenv/versions/3.8.12/envs/nlp05env/lib/python3.8/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/gilles/.pyenv/versions/3.8.12/envs/nlp05env/lib/python3.8/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/home/gilles/.pyenv/versions/3.8.12/envs/nlp05env/lib/python3.8/site-packages/keras/src/engine/training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "/home/gilles/.pyenv/versions/3.8.12/envs/nlp05env/lib/python3.8/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(f

In [14]:
# Load the IMDB dataset using TensorFlow Datasets
(ds_train, ds_test), ds_info = tfds.load('imdb_reviews', split=['train', 'test'], as_supervised=True, with_info=True)

# Split the training data into train and validation sets
def train_val_split(ds, val_size=0.2):
    ds = ds.shuffle(10000, seed=42)
    val_size = int(val_size * ds_info.splits['train'].num_examples)
    ds_val = ds.take(val_size)
    ds_train = ds.skip(val_size)
    return ds_train, ds_val

ds_train, ds_val = train_val_split(ds_train)

# Load the preprocessor and DistilBERT model from TensorFlow Hub
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2")

# Build the model
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = preprocessor(text_input)
encoder_outputs = encoder(preprocessed_text)
pooled_output = encoder_outputs['pooled_output']
output = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(pooled_output)

model = tf.keras.Model(inputs=[text_input], outputs=[output])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with batching done inside the `fit` function
epochs = 3
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# Batch size provided directly in fit function
batch_size = 16
model.fit(ds_train, validation_data=ds_val, epochs=epochs, callbacks=[early_stopping], batch_size=batch_size)

# Evaluate the model
loss, accuracy = model.evaluate(ds_test.batch(batch_size))
print(f"Test Accuracy: {accuracy:.4f}")


Epoch 1/3


ValueError: in user code:

    File "/home/gilles/.pyenv/versions/3.8.12/envs/nlp05env/lib/python3.8/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/home/gilles/.pyenv/versions/3.8.12/envs/nlp05env/lib/python3.8/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/gilles/.pyenv/versions/3.8.12/envs/nlp05env/lib/python3.8/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/home/gilles/.pyenv/versions/3.8.12/envs/nlp05env/lib/python3.8/site-packages/keras/src/engine/training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "/home/gilles/.pyenv/versions/3.8.12/envs/nlp05env/lib/python3.8/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(f